# Flower Classification - EDA & Model Training

This notebook contains:
1. **Exploratory Data Analysis (EDA)** - Understanding the flowers dataset
2. **Data Preparation** - Loading, preprocessing, augmentation
3. **Model Training** - Baseline CNN and Transfer Learning with MobileNetV2
4. **Hyperparameter Tuning** - Finding optimal settings
5. **Evaluation** - Accuracy, confusion matrix, sample predictions

**Dataset:** [TensorFlow Flowers](http://download.tensorflow.org/example_images/flower_photos.tgz)  
**Classes:** Daisy, Dandelion, Rose, Sunflower, Tulip

## Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

In [ ]:
# Paths
DATA_DIR = Path("../data/flower_photos")
MODEL_DIR = Path("../models")

# Check if data exists
if not DATA_DIR.exists():
    print("Dataset not found! Run the download script first:")
    print("  python src/download_data.py")
else:
    print(f"Dataset found at: {DATA_DIR}")

---
## 1. Exploratory Data Analysis (EDA)

### 1.1 Dataset Overview

In [ ]:
# Get class directories
class_dirs = [d for d in DATA_DIR.iterdir() if d.is_dir()]
class_names = sorted([d.name for d in class_dirs])

print(f"Number of classes: {len(class_names)}")
print(f"Classes: {class_names}")

In [ ]:
# Count images per class
class_counts = {}
for class_dir in class_dirs:
    images = list(class_dir.glob("*.jpg"))
    class_counts[class_dir.name] = len(images)

# Create DataFrame
df_counts = pd.DataFrame({
    "class": list(class_counts.keys()),
    "count": list(class_counts.values())
}).sort_values("class")

total_images = df_counts["count"].sum()
print(f"\nTotal images: {total_images}")
print("\nImages per class:")
print(df_counts.to_string(index=False))

In [ ]:
# Visualize class distribution
fig, ax = plt.subplots(figsize=(10, 5))
colors = sns.color_palette("husl", len(class_names))

bars = ax.bar(df_counts["class"], df_counts["count"], color=colors)
ax.set_xlabel("Flower Class")
ax.set_ylabel("Number of Images")
ax.set_title("Class Distribution in Flowers Dataset")

# Add count labels on bars
for bar, count in zip(bars, df_counts["count"]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
            str(count), ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

# Check for class imbalance
imbalance_ratio = df_counts["count"].max() / df_counts["count"].min()
print(f"\nClass imbalance ratio (max/min): {imbalance_ratio:.2f}")

### 1.2 Sample Images

In [ ]:
# Display sample images from each class
fig, axes = plt.subplots(len(class_names), 5, figsize=(15, 3*len(class_names)))

for i, class_name in enumerate(sorted(class_names)):
    class_path = DATA_DIR / class_name
    images = list(class_path.glob("*.jpg"))[:5]
    
    for j, img_path in enumerate(images):
        img = plt.imread(img_path)
        axes[i, j].imshow(img)
        axes[i, j].axis("off")
        if j == 0:
            axes[i, j].set_title(class_name.upper(), fontsize=12, fontweight='bold')

plt.suptitle("Sample Images from Each Class", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 1.3 Image Properties Analysis

In [ ]:
# Analyze image sizes
from PIL import Image

widths, heights = [], []
aspect_ratios = []

for class_dir in class_dirs:
    for img_path in list(class_dir.glob("*.jpg"))[:100]:  # Sample 100 per class
        with Image.open(img_path) as img:
            w, h = img.size
            widths.append(w)
            heights.append(h)
            aspect_ratios.append(w / h)

print("Image Size Statistics:")
print(f"  Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.0f}")
print(f"  Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.0f}")
print(f"  Aspect Ratio - Mean: {np.mean(aspect_ratios):.2f}")

In [ ]:
# Plot image size distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(widths, bins=30, alpha=0.7, label="Width", color="steelblue")
axes[0].hist(heights, bins=30, alpha=0.7, label="Height", color="coral")
axes[0].set_xlabel("Pixels")
axes[0].set_ylabel("Frequency")
axes[0].set_title("Image Dimensions Distribution")
axes[0].legend()

axes[1].scatter(widths, heights, alpha=0.3, s=10)
axes[1].set_xlabel("Width (pixels)")
axes[1].set_ylabel("Height (pixels)")
axes[1].set_title("Width vs Height")
axes[1].axhline(y=224, color='r', linestyle='--', label='Target size (224)')
axes[1].axvline(x=224, color='r', linestyle='--')
axes[1].legend()

plt.tight_layout()
plt.show()

### 1.4 EDA Summary

**Key Findings:**
- TODO: Add findings after running EDA
- Class distribution
- Image quality observations
- Potential challenges

---
## 2. Data Preparation

In [ ]:
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

In [ ]:
# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes: {class_names}")

In [ ]:
# Optimize dataset performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

# Visualize augmentation
for images, _ in train_ds.take(1):
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    
    # Original image
    original = images[0]
    axes[0, 0].imshow(original.numpy().astype("uint8"))
    axes[0, 0].set_title("Original")
    axes[0, 0].axis("off")
    
    # Augmented versions
    for i in range(1, 8):
        augmented = data_augmentation(tf.expand_dims(original, 0))
        row, col = divmod(i, 4)
        axes[row, col].imshow(augmented[0].numpy().astype("uint8"))
        axes[row, col].set_title(f"Augmented {i}")
        axes[row, col].axis("off")
    
    plt.suptitle("Data Augmentation Examples")
    plt.tight_layout()
    plt.show()

---
## 3. Model Training

### 3.1 Baseline Model (Simple CNN)

In [ ]:
# Build baseline CNN
baseline_model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=IMG_SIZE + (3,)),
    
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

baseline_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

baseline_model.summary()

In [ ]:
# Train baseline model
baseline_history = baseline_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

### 3.2 Transfer Learning (MobileNetV2)

In [ ]:
# Load pre-trained MobileNetV2
base_model = keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base model
base_model.trainable = False

print(f"Base model layers: {len(base_model.layers)}")

In [ ]:
# Build transfer learning model
preprocess_input = keras.applications.mobilenet_v2.preprocess_input

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

transfer_model = keras.Model(inputs, outputs)

transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

transfer_model.summary()

In [ ]:
# Train transfer learning model
transfer_history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

### 3.3 Fine-Tuning

In [ ]:
# Unfreeze top layers of base model for fine-tuning
base_model.trainable = True

# Freeze all layers except the last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with lower learning rate
transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Trainable layers: {sum([1 for l in transfer_model.layers if l.trainable])}")

In [ ]:
# Fine-tune
fine_tune_history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

---
## 4. Model Comparison & Evaluation

In [ ]:
# Plot training history comparison
def plot_history(history, title):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train')
    axes[0].plot(history.history['val_accuracy'], label='Validation')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title(f'{title} - Accuracy')
    axes[0].legend()
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train')
    axes[1].plot(history.history['val_loss'], label='Validation')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].set_title(f'{title} - Loss')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

plot_history(baseline_history, "Baseline CNN")
plot_history(transfer_history, "MobileNetV2 Transfer Learning")

In [ ]:
# Final evaluation
print("Final Evaluation on Validation Set:")
print("=" * 40)

baseline_loss, baseline_acc = baseline_model.evaluate(val_ds, verbose=0)
print(f"Baseline CNN:         {baseline_acc:.2%}")

transfer_loss, transfer_acc = transfer_model.evaluate(val_ds, verbose=0)
print(f"MobileNetV2 (tuned):  {transfer_acc:.2%}")

In [ ]:
# Confusion matrix for best model
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = transfer_model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Normalized)')
plt.tight_layout()
plt.show()

In [ ]:
# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

---
## 5. Save Model

In [ ]:
# Save the best model
MODEL_DIR.mkdir(parents=True, exist_ok=True)

model_path = MODEL_DIR / "flower_classifier.keras"
transfer_model.save(model_path)
print(f"Model saved to: {model_path}")

# Save class names
classes_path = MODEL_DIR / "class_names.txt"
with open(classes_path, 'w') as f:
    for name in class_names:
        f.write(f"{name}\n")
print(f"Class names saved to: {classes_path}")

---
## 6. Sample Predictions

In [ ]:
# Show sample predictions
for images, labels in val_ds.take(1):
    predictions = transfer_model.predict(images, verbose=0)
    
    fig, axes = plt.subplots(3, 4, figsize=(14, 10))
    axes = axes.flatten()
    
    for i in range(12):
        pred_idx = np.argmax(predictions[i])
        pred_class = class_names[pred_idx]
        true_class = class_names[labels[i]]
        confidence = predictions[i][pred_idx]
        
        color = 'green' if pred_class == true_class else 'red'
        
        axes[i].imshow(images[i].numpy().astype('uint8'))
        axes[i].set_title(f"Pred: {pred_class} ({confidence:.1%})\nTrue: {true_class}",
                          color=color, fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle("Sample Predictions (Green=Correct, Red=Wrong)", fontsize=12)
    plt.tight_layout()
    plt.show()

---
## Summary

| Model | Validation Accuracy | Notes |
|-------|---------------------|-------|
| Baseline CNN | ~X% | Simple architecture |
| MobileNetV2 (frozen) | ~X% | Feature extraction only |
| MobileNetV2 (fine-tuned) | ~X% | Best performance |

**Selected Model:** MobileNetV2 with fine-tuning

**Next Steps:**
1. Export to train.py script ✓
2. Build Flask prediction service ✓
3. Containerize with Docker
4. Deploy to cloud (optional)